<font size=6>
    <b>Model_Training_with_BERT.ipynb:</b>
    <p>Use Text Extensions for Pandas to integrate BERT tokenization with model training for named entity recognition on Pandas.</p>
</font>

# Introduction

This notebook shows how to use the open source library [Text Extensions for Pandas](https://github.com/CODAIT/text-extensions-for-pandas) to seamlessly integrate BERT tokenization and embeddings with model training for named entity recognition using [Pandas](https://pandas.pydata.org/) DataFrames.

This example will build on the analysis of the [CoNLL-2003](https://www.clips.uantwerpen.be/conll2003/ner/) corpus done in [Analyze_Model_Outputs](./Analyze_Model_Outputs.ipynb) to train a new model for named entity recognition (NER) using state-of-the-art natural language understanding with BERT tokenization and embeddings. While the model used is rather simple and will only get modest scoring results, the purpose is to demonstrate how Text Extensions for Pandas integrates BERT from [Huggingface Transformers](https://huggingface.co/transformers/index.html) with the `TensorArray` extension for model training and scoring, all within Pandas DataFrames. See [Text_Extension_for_Pandas_Overview](./Text_Extension_for_Pandas_Overview.ipynb) for `TensorArray` specification and more example usage.

The notebook is divided into the following steps:

1. Retokenize the entire corpus using a "BERT-compatible" tokenizer, and map the token/entity labels from the original corpus on to the new tokenization.
1. Generate BERT embeddings for every token in the entire corpus in one pass, and store those embeddings in a DataFrame column (of type TensorDtype) alongside the tokens and labels.
1. Persist the DataFrame with computed BERT embeddings to disk as a checkpoint.
1. Use the embeddings to train a multinomial logistic regression model to perform named entity recognition.
1. Compute precision/recall for the model predictions on a test set.


## Environment Setup

This notebook requires a Python 3.7 or later environment with NumPy, Pandas, scikit-learn, PyTorch and Huggingface `transformers`. 

The notebook also requires the  `text_extensions_for_pandas` library. You can satisfy this dependency in two ways:

* Run `pip install text_extensions_for_pandas` before running this notebook. This command adds the library to your Python environment.
* Run this notebook out of your local copy of the Text Extensions for Pandas project's [source tree](https://github.com/CODAIT/text-extensions-for-pandas). In this case, the notebook will use the version of Text Extensions for Pandas in your local source tree **if the package is not installed in your Python environment**.

In [1]:
import gc
import os
import sys
from typing import *
import numpy as np
import pandas as pd
import sklearn.pipeline
import sklearn.linear_model
import transformers

# And of course we need the text_extensions_for_pandas library itself.
try:
    import text_extensions_for_pandas as tp
except ModuleNotFoundError as e:
    # If we're running from within the project source tree and the parent Python
    # environment doesn't have the text_extensions_for_pandas package, use the
    # version in the local source tree.
    if not os.getcwd().endswith("notebooks"):
        raise e
    if ".." not in sys.path:
        sys.path.insert(0, "..")
    import text_extensions_for_pandas as tp

# Named Entity Recognition with BERT on CoNLL-2003

[CoNLL](https://www.conll.org/), the SIGNLL Conference on Computational Natural Language Learning, is an annual academic conference for natural language processing researchers. Each year's conference features a competition involving a challenging NLP task. The task for the 2003 competition involved identifying mentions of [named entities](https://en.wikipedia.org/wiki/Named-entity_recognition) in English and German news articles from the late 1990's. The corpus for this 2003 competition is one of the most widely-used benchmarks for the performance of named entity recognition models. Current [state-of-the-art results](https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003) on this corpus produce an F1 score (harmonic mean of precision and recall) of 0.93. The best F1 score in the original competition was 0.89.

For more information about this data set, we recommend reading the conference paper about the competition results, ["Introduction to the CoNLL-2003 Shared Task: Language-Independent Named Entity Recognition,"](https://www.aclweb.org/anthology/W03-0419/).

**Note that the data set is licensed for research use only. Be sure to adhere to the terms of the license when using this data set!**

The developers of the CoNLL-2003 corpus defined a file format for the corpus, based on the file format used in the earlier [Message Understanding Conference](https://en.wikipedia.org/wiki/Message_Understanding_Conference) competition. This format is generally known as "CoNLL format" or "CoNLL-2003 format".

In the following cell, we use the facilities of Text Extensions for Pandas to download a copy of the CoNLL-2003 data set. Then we read the CoNLL-2003-format file containing the `test` fold of the corpus and translate the data into a collection of Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) objects, one Dataframe per document. Finally, we display the Dataframe for the first document of the `test` fold of the corpus.

In [2]:
# Download and cache the data set.
# NOTE: This data set is licensed for research use only. Be sure to adhere
#  to the terms of the license when using this data set!
data_set_info = tp.io.conll.maybe_download_conll_data("outputs")
data_set_info

{'train': 'outputs/eng.train',
 'dev': 'outputs/eng.testa',
 'test': 'outputs/eng.testb'}

## Show how to retokenize with a BERT tokenizer.

The BERT model is originally from the paper [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) by Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova. The model is pre-trained with masked language modeling and next sentence prediction objectives, which make it effective for masked token prediction and NLU. 

With the CoNLL-2003 corpus loaded, it will need to be retokenized using a "BERT-compatible" tokenizer. Then we can map the token/entity labels from the original corpus on to the new tokenization.

We will start by showing the retokenizing process for a single document before doing the same on the entire corpus.

In [3]:
# Read in the corpus in its original tokenization.
corpus_raw = {}
for fold_name, file_name in data_set_info.items():
    df_list = tp.io.conll.conll_2003_to_dataframes(file_name, 
                                                   ["pos", "phrase", "ent"],
                                                   [False, True, True])
    corpus_raw[fold_name] = [
        df.drop(columns=["pos", "phrase_iob", "phrase_type"])
        for df in df_list
    ]

test_raw = corpus_raw["test"]

# Pick out the dataframe for a single example document.
example_df = test_raw[5]
example_df

,span,ent_iob,ent_type,sentence,line_num
0,"[0, 10): '-DOCSTART-'",O,None,"[0, 10): '-DOCSTART-'",1469
1,"[11, 18): 'CRICKET'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1471
2,"[18, 19): '-'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1472
3,"[20, 28): 'PAKISTAN'",B,LOC,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1473
4,"[29, 30): 'V'",O,None,"[11, 62): 'CRICKET- PAKISTAN V NEW ZEALAND ONE...",1474
...,...,...,...,...,...
350,"[1620, 1621): '8'",O,None,"[1590, 1634): 'Third one-day match: December 8...",1865
351,"[1621, 1622): ','",O,None,"[1590, 1634): 'Third one-day match: December 8...",1866
352,"[1623, 1625): 'in'",O,None,"[1590, 1634): 'Third one-day match: December 8...",1867
353,"[1626, 1633): 'Karachi'",B,LOC,"[1590, 1634): 'Third one-day match: December 8...",1868


The `example_df` contains columns `span` and `sentence` of dtypes `SpanDtype` and `TokenSpanDtype`. These represent spans from the target text, and here they contain tokens of the text and the sentence containing that token. See the notebook [Text_Extension_for_Pandas_Overview](./Text_Extension_for_Pandas_Overview.ipynb) for more on `SpanArray` and `TokenSpanArray`.

In [4]:
example_df.dtypes

span             SpanDtype
ent_iob             object
ent_type            object
sentence    TokenSpanDtype
line_num             int64
dtype: object

### Convert IOB-Tagged Data to Lists of Entity Mentions

The data we've looked at so far has been in [IOB2 format](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)). 
Each row of our DataFrame represents a token, and each token is tagged with an entity type (`ent_type`) and an IOB tag (`ent_iob`). The first token of each named entity mention is tagged `B`, while subsequent tokens are tagged `I`. Tokens that aren't part of any named entity are tagged `O`.

IOB2 format is a convenient way to represent a corpus, but it is a less useful representation for analyzing the result quality of named entity recognition models. Most tokens in a typical NER corpus will be tagged `O`, any measure of error rate in terms of tokens will over-emphasizing the tokens that are part of entities. Token-level error rate implicitly assigns higher weight to named entity mentions that consist of multiple tokens, further unbalancing error metrics. And most crucially, a naive comparison of IOB tags can result in marking an incorrect answer as correct. Consider a case where the correct sequence of labels is `B, B, I` but the model has output `B, I, I`; in this case, last two tokens of model output are both incorrect (the model has assigned them to the same entity as the first token), but a naive token-level comparison will consider the last token to be correct.

The CoNLL 2003 competition used the number of errors in extracting *entire* entity mentions to measure the result quality of the entries. We will use the same metric in this notebook. To compute entity-level errors, we convert the IOB-tagged tokens into pairs of `<entity span,  entity type>`. 
Text Extensions for Pandas includes a function `iob_to_spans()` that will handle this conversion for you.

In [5]:
# Convert the corpus IOB2 tagged DataFrame to one with entity span and type columns.
spans_df = tp.io.conll.iob_to_spans(example_df)
spans_df

,span,ent_type
0,"[20, 28): 'PAKISTAN'",LOC
1,"[31, 42): 'NEW ZEALAND'",LOC
2,"[80, 83): 'GMT'",MISC
3,"[85, 92): 'SIALKOT'",LOC
4,"[94, 102): 'Pakistan'",LOC
...,...,...
69,"[1488, 1501): 'Shahid Afridi'",PER
70,"[1512, 1523): 'Salim Malik'",PER
71,"[1535, 1545): 'Ijaz Ahmad'",PER
72,"[1565, 1573): 'Pakistan'",LOC


### Initialize our BERT Tokenizer and Model

Here we configure and initialize the [Huggingface transformers BERT tokenizer and model](https://huggingface.co/transformers/model_doc/bert.html). Text Extensions for Pandas provides a `make_bert_tokens()` function that will use the tokenizer to create BERT tokens as a span column in a DataFrame, suitable to compute BERT embeddings with.

In [6]:
# Huggingface transformers BERT Configuration.
bert_model_name = "dslim/bert-base-NER"

tokenizer = transformers.BertTokenizerFast.from_pretrained(bert_model_name, 
                                                           add_special_tokens=True)

# Retokenize the document's text with the BERT tokenizer as a DataFrame 
# with a span column.
bert_toks_df = tp.io.bert.make_bert_tokens(example_df["span"].values[0].target_text, tokenizer)
bert_toks_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask
0,0,"[0, 0): ''",101,0,1,True
1,1,"[0, 1): '-'",118,0,1,False
2,2,"[1, 2): 'D'",141,0,1,False
3,3,"[2, 4): 'OC'",9244,0,1,False
4,4,"[4, 6): 'ST'",9272,0,1,False
...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False
685,685,"[1623, 1625): 'in'",1107,0,1,False
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False
687,687,"[1633, 1634): '.'",119,0,1,False


In [7]:
# BERT tokenization includes special zero-length tokens.
bert_toks_df[bert_toks_df["special_tokens_mask"]]

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask
0,0,"[0, 0): ''",101,0,1,True
688,688,"[0, 0): ''",102,0,1,True


In [8]:
# Align the BERT tokens with the original tokenization.
bert_spans = tp.TokenSpanArray.align_to_tokens(bert_toks_df["span"],
                                               spans_df["span"])
pd.DataFrame({
    "original_span": spans_df["span"],
    "bert_spans": bert_spans,
    "ent_type": spans_df["ent_type"]
})

,original_span,bert_spans,ent_type
0,"[20, 28): 'PAKISTAN'","[20, 28): 'PAKISTAN'",LOC
1,"[31, 42): 'NEW ZEALAND'","[31, 42): 'NEW ZEALAND'",LOC
2,"[80, 83): 'GMT'","[80, 83): 'GMT'",MISC
3,"[85, 92): 'SIALKOT'","[85, 92): 'SIALKOT'",LOC
4,"[94, 102): 'Pakistan'","[94, 102): 'Pakistan'",LOC
...,...,...,...
69,"[1488, 1501): 'Shahid Afridi'","[1488, 1501): 'Shahid Afridi'",PER
70,"[1512, 1523): 'Salim Malik'","[1512, 1523): 'Salim Malik'",PER
71,"[1535, 1545): 'Ijaz Ahmad'","[1535, 1545): 'Ijaz Ahmad'",PER
72,"[1565, 1573): 'Pakistan'","[1565, 1573): 'Pakistan'",LOC


In [9]:
# Generate IOB2 tags and entity labels that align with the BERT tokens.
# See https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)
bert_toks_df[["ent_iob", "ent_type"]] = tp.io.conll.spans_to_iob(bert_spans, 
                                                        spans_df["ent_type"])
bert_toks_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type
0,0,"[0, 0): ''",101,0,1,True,O,<NA>
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>
...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>


In [10]:
# Create a Pandas categorical type for consistent encoding of categories
# across all documents.
ENTITY_TYPES = ["LOC", "MISC", "ORG", "PER"]
token_class_dtype, int_to_label, label_to_int = tp.io.conll.make_iob_tag_categories(ENTITY_TYPES)
token_class_dtype

CategoricalDtype(categories=['O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC',
                  'I-ORG', 'I-PER'],
                 ordered=False)

In [11]:
# The traditional way to transform NER to token classification is to 
# treat each combination of {I,O,B} X {entity type} as a different
# class. Generate class labels in that format.
classes_df = tp.io.conll.add_token_classes(bert_toks_df, token_class_dtype)
classes_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0
...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0


## Show how to compute BERT embeddings

We are going to use the BERT embeddings as the feature vector to train our model. First, we will show how they are computed 

In [12]:
# Initialize the BERT model that will be used to generate embeddings.
bert = transformers.BertModel.from_pretrained(bert_model_name)

# Force garbage collection in case this notebook is running on a low-RAM environment.
gc.collect()

# Compute BERT embeddings with the BERT model and add result to our example DataFrame.
embeddings_df = tp.io.bert.add_embeddings(classes_df, bert)
embeddings_df

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-8.30710456e-02 -3.59590381e-01 1.01506817e+0...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-2.28625804e-01 -4.93136168e-01 1.28423214e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[ 2.84803156e-02 -1.78742439e-01 1.54320931e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.46517614 -0.29836047 1.0737677 -0.0316480...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-1.07308246e-01 -3.37210238e-01 1.22697937e+0...
...,...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0,[-1.28065944e-01 -2.32436135e-03 6.78131700e-0...
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0,[ 0.30534083 -0.526257 0.8281703 -0.2741487...
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1,[-0.04873935 -0.33797342 -0.05835137 0.7557765...
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0,[-5.28975204e-03 -2.97430754e-01 7.16174066e-0...


In [13]:
# The `embedding` column is an extension type `TensorDtype` that holds a 
#`TensorArray` provided by Text Extensions for Pandas.
embeddings_df["embedding"].dtype

A `TensorArray` can be constructed with a NumPy array of arbitrary dimensions, added to a DataFrame, then used with standard Pandas functionality. See the notebook [Text_Extension_for_Pandas_Overview](./Text_Extensions_for_Pandas.ipynb) for more on `TensorArray`.

In [14]:
# Zero-copy conversion to NumPy can be done by first unwrapping the
# `TensorArray` with `.array` and calling `to_numpy()`.
embeddings_arr = embeddings_df["embedding"].array.to_numpy()
embeddings_arr.dtype, embeddings_arr.shape

(dtype('float32'), (689, 768))

## Generate BERT tokens and BERT embeddings for the entire corpus

Text Extensions for Pandas has a convenience function that will combine the above cells to create BERT tokens and embeddings. We will use this to add embeddings to the entire corpus.

In [15]:
# Example usage of the convenience function to create BERT tokens and embeddings.
tp.io.bert.conll_to_bert(example_df, tokenizer, bert, token_class_dtype)

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-8.30710456e-02 -3.59590381e-01 1.01506817e+0...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-2.28625804e-01 -4.93136168e-01 1.28423214e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[ 2.84803156e-02 -1.78742439e-01 1.54320931e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.46517614 -0.29836047 1.0737677 -0.0316480...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-1.07308246e-01 -3.37210238e-01 1.22697937e+0...
...,...,...,...,...,...,...,...,...,...,...,...
684,684,"[1621, 1622): ','",117,0,1,False,O,<NA>,O,0,[-1.28065944e-01 -2.32436135e-03 6.78131700e-0...
685,685,"[1623, 1625): 'in'",1107,0,1,False,O,<NA>,O,0,[ 0.30534083 -0.526257 0.8281703 -0.2741487...
686,686,"[1626, 1633): 'Karachi'",16237,0,1,False,B,LOC,B-LOC,1,[-0.04873935 -0.33797342 -0.05835137 0.7557765...
687,687,"[1633, 1634): '.'",119,0,1,False,O,<NA>,O,0,[-5.28975204e-03 -2.97430754e-01 7.16174066e-0...


When this notebook is running on a resource-constrained environment like [Binder](https://mybinder.org/),
there may not be enough RAM available to hold all the embeddings in memory.
So we use [Gaussian random projection](https://scikit-learn.org/stable/modules/random_projection.html#gaussian-random-projection) to reduce the size of the embeddings.
The projection shrinks the embeddings by a factor of 3 at the expense of a small
decrease in model accuracy.

Change the constant `SHRINK_EMBEDDINGS` in the following cell to `False` if you want to disable this behavior.

In [16]:
SHRINK_EMBEDDINGS = True
PROJECTION_DIMS = 256
RANDOM_SEED=42

import sklearn.random_projection
projection = sklearn.random_projection.GaussianRandomProjection(
    n_components=PROJECTION_DIMS, random_state=RANDOM_SEED)

def maybe_shrink_embeddings(df):
    if SHRINK_EMBEDDINGS:
        df["embedding"] = tp.TensorArray(projection.fit_transform(df["embedding"]))
    return df

In [17]:
# Run the entire corpus through our processing pipeline.
bert_toks_by_fold = {}
for fold_name in corpus_raw.keys():
    print(f"Processing fold '{fold_name}'...")
    raw = corpus_raw[fold_name]
    bert_toks_by_fold[fold_name] = tp.jupyter.run_with_progress_bar(
        len(raw), lambda i: maybe_shrink_embeddings(tp.io.bert.conll_to_bert(
            raw[i], tokenizer, bert, token_class_dtype)))
    
bert_toks_by_fold["dev"][20]

Processing fold 'train'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=946, style=ProgressStyle(desc…

Processing fold 'dev'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=216, style=ProgressStyle(desc…

Processing fold 'test'...


IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=231, style=ProgressStyle(desc…

,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-0.06799735 2.66429278 -0.59708109 -0.8213804...
1,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-7.26248335e-01 2.60041412e+00 -1.18024932e+0...
2,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[-9.68885853e-02 2.95125174e+00 -1.06147524e+0...
3,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.15686832 2.58594635 -0.88913219 -1.1479725...
4,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-1.36131452e-01 2.82019518e+00 -1.65665598e+0...
...,...,...,...,...,...,...,...,...,...,...,...
2154,2154,"[5704, 5705): ')'",114,0,1,False,O,<NA>,O,0,[-1.64370131 1.25760216 -0.51353996 -0.0972023...
2155,2155,"[5706, 5708): '39'",3614,0,1,False,O,<NA>,O,0,[-1.62701462 1.35135 -0.7599943 -0.0166738...
2156,2156,"[5708, 5709): '.'",119,0,1,False,O,<NA>,O,0,[-1.44683186 1.38293863 -0.67679566 -0.1660645...
2157,2157,"[5709, 5711): '93'",5429,0,1,False,O,<NA>,O,0,[-1.67463959 1.61159401 -0.77767773 -0.1787928...


## Collate the data structures we've generated so far

In [18]:
# Create a single DataFrame with the entire corpus's embeddings.
corpus_df = tp.io.conll.combine_folds(bert_toks_by_fold)
corpus_df

,fold,doc_num,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,"[0, 0): ''",101,0,1,True,O,<NA>,O,0,[-1.13115576e+00 2.76648571e+00 -1.06501381e+0...
1,train,0,1,"[0, 1): '-'",118,0,1,False,O,<NA>,O,0,[-1.22220795e+00 2.52742513e+00 -1.28222358e+0...
2,train,0,2,"[1, 2): 'D'",141,0,1,False,O,<NA>,O,0,[-7.57985410e-01 2.73181597e+00 -1.41631688e+0...
3,train,0,3,"[2, 4): 'OC'",9244,0,1,False,O,<NA>,O,0,[-0.67307861 2.38562717 -1.23673184 -0.6060617...
4,train,0,4,"[4, 6): 'ST'",9272,0,1,False,O,<NA>,O,0,[-5.52802419e-01 2.76605601e+00 -1.69859440e+0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
416536,test,230,314,"[1386, 1393): 'brother'",1711,0,1,False,O,<NA>,O,0,[-1.76998072 1.7405787 -1.01661536 -0.5360813...
416537,test,230,315,"[1393, 1394): ','",117,0,1,False,O,<NA>,O,0,[-2.21704182 1.18801414 -0.58463333 -0.2408621...
416538,test,230,316,"[1395, 1400): 'Bobby'",5545,0,1,False,B,PER,B-PER,4,[ 1.72650395e-01 2.21286938e+00 1.06825029e+0...
416539,test,230,317,"[1400, 1401): '.'",119,0,1,False,O,<NA>,O,0,[-2.02287426e+00 1.54863003e+00 -7.25010390e-0...


## Checkpoint

With the `TensorArray` from Text Extensions for Pandas, the computed embeddings can be persisted as a tensor along with the rest of the DataFrame using standard Pandas input/output methods. Since this is a costly operation and the embeddings are deterministic, it can save lots of time to checkpoint the data here and save the results to disk. This will allow us to continue working with model training without needing to re-compute the BERT embeddings again.
 
### Save DataFrame with Embeddings Tensor

In [19]:
# Write the tokenized corpus with embeddings to a Feather file.
# We can't currently serialize span columns that cover multiple documents (see issue #73 https://github.com/CODAIT/text-extensions-for-pandas/issues/73),
# so drop span columns from the contents we write to the Feather file.
cols_to_drop = [c for c in corpus_df.columns if "span" in c]
corpus_df.drop(columns=cols_to_drop).to_feather("outputs/corpus.feather")

### Load DataFrame with Previously Computed Embeddings

In [20]:
# Read the serialized embeddings back in so that you can rerun the model 
# training parts of this notebook (the cells from here onward) without 
# regenerating the embeddings.
corpus_df = pd.read_feather("outputs/corpus.feather")
corpus_df

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,101,0,1,True,O,<NA>,O,0,[-1.13115576e+00 2.76648571e+00 -1.06501381e+0...
1,train,0,1,118,0,1,False,O,<NA>,O,0,[-1.22220795e+00 2.52742513e+00 -1.28222358e+0...
2,train,0,2,141,0,1,False,O,<NA>,O,0,[-7.57985410e-01 2.73181597e+00 -1.41631688e+0...
3,train,0,3,9244,0,1,False,O,<NA>,O,0,[-0.67307861 2.38562717 -1.23673184 -0.6060617...
4,train,0,4,9272,0,1,False,O,<NA>,O,0,[-5.52802419e-01 2.76605601e+00 -1.69859440e+0...
...,...,...,...,...,...,...,...,...,...,...,...,...
416536,test,230,314,1711,0,1,False,O,<NA>,O,0,[-1.76998072 1.7405787 -1.01661536 -0.5360813...
416537,test,230,315,117,0,1,False,O,<NA>,O,0,[-2.21704182 1.18801414 -0.58463333 -0.2408621...
416538,test,230,316,5545,0,1,False,B,PER,B-PER,4,[ 1.72650395e-01 2.21286938e+00 1.06825029e+0...
416539,test,230,317,119,0,1,False,O,<NA>,O,0,[-2.02287426e+00 1.54863003e+00 -7.25010390e-0...


## Training a model on the BERT embeddings

Now we will use the loaded BERT embeddings to train a multinomial model to predict the token class from the embeddings tensor.

In [21]:
# Extract the training set DataFrame.
train_df = corpus_df[corpus_df["fold"] == "train"]
train_df

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding
0,train,0,0,101,0,1,True,O,<NA>,O,0,[-1.13115576e+00 2.76648571e+00 -1.06501381e+0...
1,train,0,1,118,0,1,False,O,<NA>,O,0,[-1.22220795e+00 2.52742513e+00 -1.28222358e+0...
2,train,0,2,141,0,1,False,O,<NA>,O,0,[-7.57985410e-01 2.73181597e+00 -1.41631688e+0...
3,train,0,3,9244,0,1,False,O,<NA>,O,0,[-0.67307861 2.38562717 -1.23673184 -0.6060617...
4,train,0,4,9272,0,1,False,O,<NA>,O,0,[-5.52802419e-01 2.76605601e+00 -1.69859440e+0...
...,...,...,...,...,...,...,...,...,...,...,...,...
281104,train,945,53,17057,0,1,False,B,ORG,B-ORG,3,[-1.36448985e+00 1.38776617e-01 -1.22179374e-0...
281105,train,945,54,122,0,1,False,O,<NA>,O,0,[-1.45446814e+00 1.42937287e+00 -7.52059640e-0...
281106,train,945,55,4617,0,1,False,B,ORG,B-ORG,3,[-1.0318755 0.40648041 0.093575 -1.8624162...
281107,train,945,56,123,0,1,False,O,<NA>,O,0,[-1.25979042e+00 1.39594314e+00 -8.63471313e-0...


In [22]:
# Train a multinomial logistic regression model on the training set.
MULTI_CLASS = "multinomial"
    
# How many iterations to run the BGFS optimizer when fitting logistic
# regression models. 100 ==> Fast; 10000 ==> Full convergence
LBGFS_ITERATIONS = 10000

base_pipeline = sklearn.pipeline.Pipeline([
    # Standard scaler. This only makes a difference for certain classes
    # of embeddings.
    #("scaler", sklearn.preprocessing.StandardScaler()),
    ("mlogreg", sklearn.linear_model.LogisticRegression(
        multi_class=MULTI_CLASS,
        verbose=10,
        max_iter=LBGFS_ITERATIONS
    ))
])

X_train = train_df["embedding"].values
Y_train = train_df["token_class_id"]
base_model = base_pipeline.fit(X_train, Y_train)
base_model

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 12.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 12.1min finished


Pipeline(steps=[('mlogreg',
                 LogisticRegression(max_iter=10000, multi_class='multinomial',
                                    verbose=10))])

## Make Predictions on Token Class from BERT Embeddings

Using our model, we can now predict the token class from the test set using the computed embeddings.

In [23]:
# Define a function that will let us make predictions on a fold of the corpus.
def predict_on_df(df: pd.DataFrame, id_to_class: Dict[int, str], predictor):
    """
    Run a trained model on a DataFrame of tokens with embeddings.

    :param df: DataFrame of tokens for a document, containing a TokenSpan column
     called "embedding" for each token.
    :param id_to_class: Mapping from class ID to class name, as returned by
     :func:`text_extensions_for_pandas.make_iob_tag_categories`
    :param predictor: Python object with a `predict_proba` method that accepts
     a numpy array of embeddings.
    :returns: A copy of `df`, with the following additional columns:
     `predicted_id`, `predicted_class`, `predicted_iob`, `predicted_type`
     and `predicted_class_pr`.
    """
    result_df = df.copy()
    class_pr = tp.TensorArray(predictor.predict_proba(result_df["embedding"]))
    result_df["predicted_id"] = np.argmax(class_pr, axis=1)
    result_df["predicted_class"] = [id_to_class[i]
                                    for i in result_df["predicted_id"].values]
    iobs, types = tp.io.conll.decode_class_labels(result_df["predicted_class"].values)
    result_df["predicted_iob"] = iobs
    result_df["predicted_type"] = types
    result_df["predicted_class_pr"] = class_pr
    return result_df

In [24]:
# Make predictions on the test set.
test_results_df = predict_on_df(corpus_df[corpus_df["fold"] == "test"], int_to_label, base_model)
test_results_df.head()

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding,predicted_id,predicted_class,predicted_iob,predicted_type,predicted_class_pr
351001,test,0,0,101,0,1,True,O,<NA>,O,0,[ 7.41897173e-02 2.81491878e+00 -8.00371801e-0...,0,O,O,None,[9.99820136e-01 5.76411557e-07 1.32056564e-05 4...
351002,test,0,1,118,0,1,False,O,<NA>,O,0,[-7.55312514e-01 2.71243455e+00 -1.26390291e+0...,0,O,O,None,[9.98759962e-01 2.48538225e-07 3.29359436e-06 1...
351003,test,0,2,141,0,1,False,O,<NA>,O,0,[ 1.14652583e-01 3.11397891e+00 -9.88389156e-0...,0,O,O,None,[9.97874371e-01 2.93584601e-04 1.98340060e-04 4...
351004,test,0,3,9244,0,1,False,O,<NA>,O,0,[-0.14387768 2.92576727 -0.8995262 -1.0219667...,0,O,O,None,[9.99435525e-01 5.45531272e-10 5.25556180e-08 4...
351005,test,0,4,9272,0,1,False,O,<NA>,O,0,[ 8.37605661e-02 3.06716262e+00 -1.82881357e+0...,0,O,O,None,[9.99799247e-01 9.90640372e-10 5.57111176e-08 7...


In [25]:
# Take a slice to show a region with more entities.
test_results_df.iloc[40:50]

,fold,doc_num,token_id,input_id,token_type_id,attention_mask,special_tokens_mask,ent_iob,ent_type,token_class,token_class_id,embedding,predicted_id,predicted_class,predicted_iob,predicted_type,predicted_class_pr
351041,test,0,40,3309,0,1,False,I,PER,I-PER,8,[ 6.02822922e-02 2.83345047e+00 -9.44222127e-0...,5,I-LOC,I,LOC,[2.42910258e-02 2.07962799e-03 1.58510840e-02 1...
351042,test,0,41,1306,0,1,False,I,PER,I-PER,8,[ 0.01181527 2.48048865 -1.34062274 -0.6055012...,5,I-LOC,I,LOC,[1.82555563e-01 1.57493427e-01 6.96706235e-07 3...
351043,test,0,42,2001,0,1,False,I,PER,I-PER,8,[ 1.89673285e-01 2.08413903e+00 -1.80942578e+0...,5,I-LOC,I,LOC,[2.75929233e-04 7.30213318e-03 1.39423273e-06 7...
351044,test,0,43,1181,0,1,False,I,PER,I-PER,8,[-0.08919204 2.67304318 -1.39639924 -0.3512777...,5,I-LOC,I,LOC,[1.08460873e-02 8.70060041e-03 7.36007663e-06 1...
351045,test,0,44,2293,0,1,False,I,PER,I-PER,8,[-0.56755931 2.19156102 -1.92558757 -0.9504741...,5,I-LOC,I,LOC,[9.38443095e-02 4.20631375e-02 9.92966749e-06 7...
351046,test,0,45,18589,0,1,False,B,LOC,B-LOC,1,[-2.57563863e-02 2.41765662e+00 -1.77096046e+0...,1,B-LOC,B,LOC,[9.64693806e-04 7.55420115e-01 7.55352933e-07 1...
351047,test,0,46,118,0,1,False,I,LOC,I-LOC,5,[-8.14390595e-01 2.24322288e+00 -1.49270455e+0...,5,I-LOC,I,LOC,[4.33464875e-01 1.89824636e-02 2.99781003e-06 8...
351048,test,0,47,19016,0,1,False,I,LOC,I-LOC,5,[-7.61381109e-01 2.10408010e+00 -1.12517440e+0...,5,I-LOC,I,LOC,[6.82181798e-02 2.40495844e-01 5.14693137e-07 1...
351049,test,0,48,2249,0,1,False,I,LOC,I-LOC,5,[-5.02346328e-01 2.46721668e+00 -1.73443730e+0...,5,I-LOC,I,LOC,[1.59023208e-03 1.27708150e-02 3.93896705e-06 3...
351050,test,0,49,117,0,1,False,O,<NA>,O,0,[-1.08983785e+00 2.48397334e+00 -1.10079260e+0...,0,O,O,None,[9.99840462e-01 7.38650674e-06 1.78692503e-06 3...


## Compute Precision and Recall

With our model predictions on the test set, we can now compute precision and recall. To do this, we will use the following steps:

1. Split up test set predictions by document, so we can work on the document level.
1. Join the test predictions with token information into one DataFrame per document.
1. Convert each DataFrame from IOB2 format to span, entity type pairs as done before.
1. Compute accuracy for each document as a DataFrame.
1. Aggregate per-document accuracy to get overal precision/recall.

In [26]:
# Split model outputs for an entire fold back into documents and add
# token information.

# Get unique documents per fold.
fold_and_doc = test_results_df[["fold", "doc_num"]] \
        .drop_duplicates() \
        .to_records(index=False)

# Index by fold, doc and token id, then make sure sorted.
indexed_df = test_results_df \
        .set_index(["fold", "doc_num", "token_id"], verify_integrity=True) \
        .sort_index()

# Join predictions with token information, for each document.
test_results_by_doc = {}
for collection, doc_num in fold_and_doc:
    doc_slice = indexed_df.loc[collection, doc_num].reset_index()
    doc_toks = bert_toks_by_fold[collection][doc_num][
        ["token_id", "span", "ent_iob", "ent_type"]
    ].rename(columns={"id": "token_id"})
    joined_df = doc_toks.copy().merge(
        doc_slice[["token_id", "predicted_iob", "predicted_type"]])
    test_results_by_doc[(collection, doc_num)] = joined_df
    
# Test results are now in one DataFrame per document.
test_results_by_doc[("test", 0)].iloc[40:60]

,token_id,span,ent_iob,ent_type,predicted_iob,predicted_type
40,40,"[68, 70): 'di'",I,PER,I,LOC
41,41,"[70, 71): 'm'",I,PER,I,LOC
42,42,"[72, 74): 'La'",I,PER,I,LOC
43,43,"[74, 75): 'd'",I,PER,I,LOC
44,44,"[75, 77): 'ki'",I,PER,I,LOC
45,45,"[78, 80): 'AL'",B,LOC,B,LOC
46,46,"[80, 81): '-'",I,LOC,I,LOC
47,47,"[81, 83): 'AI'",I,LOC,I,LOC
48,48,"[83, 84): 'N'",I,LOC,I,LOC
49,49,"[84, 85): ','",O,<NA>,O,None


In [27]:
# Convert IOB2 format to spans, entity type with `tp.io.conll.iob_to_spans()`.
test_actual_spans = {k: tp.io.conll.iob_to_spans(v) for k, v in test_results_by_doc.items()}
test_model_spans = {k:
        tp.io.conll.iob_to_spans(v, iob_col_name = "predicted_iob",
                                 entity_type_col_name = "predicted_type")
            .rename(columns={"predicted_type": "ent_type"})
        for k, v in test_results_by_doc.items()}

test_model_spans[("test", 0)].head()

,span,ent_type
0,"[19, 24): 'JAPAN'",PER
1,"[29, 34): 'LUCKY'",LOC
2,"[40, 45): 'CHINA'",LOC
3,"[49, 50): 'S'",MISC
4,"[66, 77): 'Nadim Ladki'",PER


In [28]:
# Compute per-document statistics into a single DataFrame.
test_stats_by_doc = tp.io.conll.compute_accuracy_by_document(test_actual_spans, test_model_spans)
test_stats_by_doc

,fold,doc_num,num_true_positives,num_extracted,num_entities,precision,recall,F1
0,test,0,42,47,45,0.893617,0.933333,0.913043
1,test,1,41,42,44,0.976190,0.931818,0.953488
2,test,2,51,55,54,0.927273,0.944444,0.935780
3,test,3,41,45,44,0.911111,0.931818,0.921348
4,test,4,17,19,19,0.894737,0.894737,0.894737
...,...,...,...,...,...,...,...,...
226,test,226,7,7,7,1.000000,1.000000,1.000000
227,test,227,18,19,21,0.947368,0.857143,0.900000
228,test,228,24,25,27,0.960000,0.888889,0.923077
229,test,229,26,27,27,0.962963,0.962963,0.962963


In [29]:
# Collection-wide precision and recall can be computed by aggregating
# our DataFrame.
tp.io.conll.compute_global_accuracy(test_stats_by_doc)

{'num_true_positives': 4725,
 'num_entities': 5648,
 'num_extracted': 5595,
 'precision': 0.8445040214477212,
 'recall': 0.8365793201133145,
 'F1': 0.8405229920839634}

### Adjusting the BERT Model Output

The above results aren't bad for a first shot, but taking a look a some of the predictions will show that sometimes the tokens have been split up into multiple entities. This is because the BERT tokenizer uses WordPiece to make subword tokens, see https://huggingface.co/transformers/tokenizer_summary.html and https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf for more information.

This is going to cause a problem when computing precision/recall because we are comparing exact spans, and if the entity is split, it will be counted as a false negative _and_ possibly one or more false positives. Luckily we can fix up with Text Extension for Pandas.

Let's drill down to see an example of the issue and how to correct it.

In [30]:
# Every once in a while, the BERT model will split a token in the original data
# set into multiple entities. For example, look at document 202 of the test set:
test_model_spans[("test", 202)].head(10)

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 151): 'W'",ORG
7,"[151, 156): 'idnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG


Notice `[150, 151): 'W'` and `[151, 156): 'idnes'`. These outputs are part
of the same original token, but have been split by the model.

In [31]:
# We can use spanner algebra in `tp.spanner.overlap_join()`
# to fix up these outputs.
spans_df = test_model_spans[("test", 202)]
toks_df = test_raw[202]

# First, find which tokens the spans overlap with:
overlaps_df = (
    tp.spanner.overlap_join(spans_df["span"], toks_df["span"],
                            "span", "corpus_token")
        .merge(spans_df)
)
overlaps_df.head(10)

,span,corpus_token,ent_type
0,"[11, 22): 'RUGBY UNION'","[11, 16): 'RUGBY'",ORG
1,"[11, 22): 'RUGBY UNION'","[17, 22): 'UNION'",ORG
2,"[24, 31): 'BRITISH'","[24, 31): 'BRITISH'",MISC
3,"[41, 47): 'LONDON'","[41, 47): 'LONDON'",LOC
4,"[70, 77): 'British'","[70, 77): 'British'",MISC
5,"[111, 125): 'Pilkington Cup'","[111, 121): 'Pilkington'",MISC
6,"[111, 125): 'Pilkington Cup'","[122, 125): 'Cup'",MISC
7,"[139, 146): 'Reading'","[139, 146): 'Reading'",ORG
8,"[150, 151): 'W'","[150, 156): 'Widnes'",ORG
9,"[151, 156): 'idnes'","[150, 156): 'Widnes'",ORG


In [32]:
# Next, compute the minimum span that covers all the corpus tokens
# that overlap with each entity span.
agg_df = (
    overlaps_df
    .groupby("span")
    .aggregate({"corpus_token": "sum", "ent_type": "first"})
    .reset_index()
)
agg_df.head(10)

,span,corpus_token,ent_type
0,"[11, 22): 'RUGBY UNION'","[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'","[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'","[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'","[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'","[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'","[139, 146): 'Reading'",ORG
6,"[150, 151): 'W'","[150, 156): 'Widnes'",ORG
7,"[151, 156): 'idnes'","[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'","[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'","[180, 184): 'Bath'",ORG


In [33]:
# Finally, take unique values and covert character-based spans to token
# spans in the corpus tokenization (since the new offsets might not match a
# BERT tokenizer token boundary).
cons_df = (
    tp.spanner.consolidate(agg_df, "corpus_token")[["corpus_token", "ent_type"]]
        .rename(columns={"corpus_token": "span"})
)
cons_df["span"] = tp.TokenSpanArray.align_to_tokens(toks_df["span"],
                                                    cons_df["span"])
cons_df.head(10)

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG
10,"[188, 198): 'Harlequins'",ORG


In [34]:
# Text Extensions for Pandas contains a single function that repeats the actions of the 
# previous 3 cells.
tp.io.bert.align_bert_tokens_to_corpus_tokens(test_model_spans[("test", 202)], test_raw[202]).head(10)

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG
10,"[188, 198): 'Harlequins'",ORG


In [35]:
# Run all of our DataFrames through `align_bert_tokens_to_corpus_tokens()`.
keys = list(test_model_spans.keys())
new_values = tp.jupyter.run_with_progress_bar(
    len(keys), 
    lambda i: tp.io.bert.align_bert_tokens_to_corpus_tokens(test_model_spans[keys[i]], test_raw[keys[i][1]]))
test_model_spans = {k: v for k, v in zip(keys, new_values)}
test_model_spans[("test", 202)].head(10)

IntProgress(value=0, description='Starting...', layout=Layout(width='100%'), max=231, style=ProgressStyle(desc…

,span,ent_type
0,"[11, 22): 'RUGBY UNION'",ORG
1,"[24, 31): 'BRITISH'",MISC
2,"[41, 47): 'LONDON'",LOC
3,"[70, 77): 'British'",MISC
4,"[111, 125): 'Pilkington Cup'",MISC
5,"[139, 146): 'Reading'",ORG
6,"[150, 156): 'Widnes'",ORG
8,"[159, 166): 'English'",MISC
9,"[180, 184): 'Bath'",ORG
10,"[188, 198): 'Harlequins'",ORG


In [36]:
# Compute per-document statistics into a single DataFrame.
test_stats_by_doc = tp.io.conll.compute_accuracy_by_document(test_actual_spans, test_model_spans)
test_stats_by_doc

,fold,doc_num,num_true_positives,num_extracted,num_entities,precision,recall,F1
0,test,0,43,47,45,0.914894,0.955556,0.934783
1,test,1,41,42,44,0.976190,0.931818,0.953488
2,test,2,52,54,54,0.962963,0.962963,0.962963
3,test,3,42,44,44,0.954545,0.954545,0.954545
4,test,4,17,19,19,0.894737,0.894737,0.894737
...,...,...,...,...,...,...,...,...
226,test,226,7,7,7,1.000000,1.000000,1.000000
227,test,227,18,19,21,0.947368,0.857143,0.900000
228,test,228,24,25,27,0.960000,0.888889,0.923077
229,test,229,26,27,27,0.962963,0.962963,0.962963


In [37]:
# Collection-wide precision and recall can be computed by aggregating
# our DataFrame.
tp.io.conll.compute_global_accuracy(test_stats_by_doc)

{'num_true_positives': 4871,
 'num_entities': 5648,
 'num_extracted': 5526,
 'precision': 0.8814694173000361,
 'recall': 0.8624291784702549,
 'F1': 0.8718453552890639}

These results are a bit better than before, and while the F1 score is not high compared to todays standards, it is decent enough for a simplistic model. More importantly, we did show it was fairly easy to create a model for named entity recognition and analyze the output by leveraging the functionalitiy of Pandas DataFrames along with [Text Extensions for Pandas](https://github.com/CODAIT/text-extensions-for-pandas) `SpanArray`, `TensorArray` and integration with BERT from [Huggingface Transformers](https://huggingface.co/transformers/index.html).